In [ ]:
import torch
import torch.nn as nn
from torchsummary import summary  # pip install torchsummary

class small_basic_block(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(small_basic_block, self).__init__()
        q_ch_out = ch_out // 4
        self.block = nn.Sequential(
            nn.Conv2d(ch_in, q_ch_out, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(q_ch_out, q_ch_out, kernel_size=(3, 1), padding=(1, 0)),
            nn.ReLU(),
            nn.Conv2d(q_ch_out, q_ch_out, kernel_size=(1, 3), padding=(0, 1)),
            nn.ReLU(),
            nn.Conv2d(q_ch_out, ch_out, kernel_size=1),
        )
    def forward(self, x):
        return self.block(x)

# Create model
class LPRNet(nn.Module):
    def __init__(self, class_num, dropout_rate=0.5):
        super(LPRNet, self).__init__()
        self.class_num = class_num

        self.backbone = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(), # 2
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1)),
            small_basic_block(ch_in=64, ch_out=128), # [-1, 128, 20, 90]
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(), # 6
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 1, 2)), # [-1, 64, 18, 44], use max pool 3D when we want max pool can change #channels
            small_basic_block(ch_in=64, ch_out=256),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            small_basic_block(ch_in=256, ch_out=256),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(), # 13
            nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(4, 1, 2)),
            nn.Dropout(dropout_rate),
            nn.Conv2d(in_channels=64, out_channels=256, kernel_size=(1, 4), stride=1),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Conv2d(in_channels=256, out_channels=class_num, kernel_size=(13, 1), stride=1),
            nn.BatchNorm2d(num_features=class_num),
            nn.ReLU(), # 22
        )

        self.container = nn.Sequential(
            nn.Conv2d(in_channels=448+self.class_num, out_channels=self.class_num, kernel_size=(1, 1), stride=(1, 1)),
        )
        
    def forward(self, x):
        keep_features = list()
        for i, layer in enumerate(self.backbone.children()):
            x = layer(x)
            if i in [2, 6, 13, 22]:
                keep_features.append(x)

        global_context = list()
        for i, f in enumerate(keep_features):
            if i in [0, 1]:
                f = nn.AvgPool2d(kernel_size=5, stride=5)(f)
            if i in [2]:
                f = nn.AvgPool2d(kernel_size=(4, 10), stride=(4, 2))(f)
            # normalize f
            f_pow = torch.pow(f, 2)
            f_mean = torch.mean(f_pow)
            f = torch.div(f, f_mean)
            global_context.append(f)

        x = torch.cat(global_context, 1)
        x = self.container(x)
        logits = torch.mean(x, dim=2)
        return logits
    
    def show_num_layer(self):
        for i, layer in enumerate(self.backbone.children()):
            print(f"{i}: {layer}")

model = LPRNet(class_num=37)
# Summarize
summary(model, input_size=(3, 24, 94))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 22, 92]           1,792
       BatchNorm2d-2           [-1, 64, 22, 92]             128
              ReLU-3           [-1, 64, 22, 92]               0
         MaxPool3d-4           [-1, 64, 20, 90]               0
            Conv2d-5           [-1, 32, 20, 90]           2,080
              ReLU-6           [-1, 32, 20, 90]               0
            Conv2d-7           [-1, 32, 20, 90]           3,104
              ReLU-8           [-1, 32, 20, 90]               0
            Conv2d-9           [-1, 32, 20, 90]           3,104
             ReLU-10           [-1, 32, 20, 90]               0
           Conv2d-11          [-1, 128, 20, 90]           4,224
small_basic_block-12          [-1, 128, 20, 90]               0
      BatchNorm2d-13          [-1, 128, 20, 90]             256
             ReLU-14          [-1, 128,

In [ ]:
import torch

# Batch of 2 fake "license plate images"
x = torch.randn(2, 3, 24, 94)  # (batch, channels, height, width)

model = LPRNet(class_num=37)   # same as your code
out = model(x)

print("Input shape :", x.shape)
print("Output shape:", out.shape)


Input shape : torch.Size([2, 3, 24, 94])
Output shape: torch.Size([2, 37, 18])
Output sample: tensor([[ 2.2272,  1.8685,  1.9702,  2.4631,  2.4567,  2.3155,  1.7557,  2.2795,
          2.1995,  2.1161,  2.0995,  2.1658,  2.1669,  2.3838,  1.9219,  2.3758,
          1.9487,  2.3693],
        [-0.6699,  0.0177,  0.3519,  0.2832,  0.6932,  0.7621,  0.0287, -0.4655,
         -0.6731, -0.2783,  0.3519,  0.2196, -0.0034, -0.2305, -0.0450, -0.1931,
          0.1755,  0.0480],
        [ 0.5912,  0.6493,  0.9425,  0.3298,  0.2324,  0.5261,  1.1269,  0.3062,
          0.4118,  0.2523,  0.5166,  0.6420,  0.0594,  0.2190,  0.8406,  0.3382,
          0.0389, -0.1020],
        [ 0.4167,  0.5450,  0.4984,  0.6369,  1.0176,  1.1781,  0.5243,  0.4525,
          0.5488,  0.7714,  0.2684,  0.7552,  0.6098,  0.5767,  0.7253,  0.5358,
          0.8181,  0.6244],
        [-0.8051, -0.8604, -1.4445, -1.3570, -1.4515, -1.4173, -0.9144, -0.8844,
         -0.9152, -1.0738, -0.9001, -0.8833, -0.7006, -0.8877, -0